In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

23/07/18 11:47:14 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.242.128 instead (on interface ens33)
23/07/18 11:47:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/18 11:47:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_tweet = spark.read.json("/home/eugenio/Documents/curso/datalake/Silver/twitter_datascience/tweets")

In [5]:
df_tweet.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|process_date|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+------------+
|       42|             22|2023-07-16T16:31:...| 48|        78|         18|         55|           82|Tweet fictício ge...|  2023-07-16|
|       46|              6|2023-07-16T14:07:...| 65|        59|         69|         80|           35|Tweet fictício ge...|  2023-07-16|
|       94|             19|2023-07-16T00:37:...| 96|         3|         95|         66|           98|Tweet fictício cr...|  2023-07-16|
|       27|             99|2023-07-16T02:28:...| 95|        59|         17|         99|           45|Um terceiro tweet...|  2023-07-16|
|       56|             26|2023-07-16T03:06:...|

In [6]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [7]:
from pyspark.sql import functions as f

In [9]:
tweet_conversas = df_tweet.alias('tweet')\
                    .groupBy(f.to_date("created_at").alias('created_date'))\
                    .agg(
                        f.countDistinct("author_id").alias('n_tweets'),
                        f.sum("like_count").alias('n_likes'),
                        f.sum("quote_count").alias('n_quote'),
                        f.sum("reply_count").alias('n_reply'),
                        f.sum("retweet_count").alias('retweet')                        
                    ).withColumn("Weekday", f.date_format("created_date", "E"))

In [10]:
tweet_conversas.show()

+------------+--------+-------+-------+-------+-------+-------+
|created_date|n_tweets|n_likes|n_quote|n_reply|retweet|Weekday|
+------------+--------+-------+-------+-------+-------+-------+
|  2023-07-15|      10|    408|    421|    582|    489|    Sat|
|  2023-07-12|       9|    591|    566|    451|    338|    Wed|
|  2023-07-13|       9|    370|    460|    411|    405|    Thu|
|  2023-07-16|      19|   1092|   1238|   1213|   1061|    Sun|
|  2023-07-14|       9|    585|    301|    536|    478|    Fri|
|  2023-07-17|      10|    434|    521|    374|    539|    Mon|
+------------+--------+-------+-------+-------+-------+-------+



In [15]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [17]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [18]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       87|             43|2023-07-14T15:59:...| 44|        18|         96|         28|           88|Este é um tweet f...|
|       30|             85|2023-07-14T02:22:...| 46|        77|         78|         36|           34|Outro tweet fictí...|
|       61|             55|2023-07-14T06:55:...| 88|        95|         37|         99|           54|Outro tweet fictí...|
|       35|             44|2023-07-14T14:32:...| 75|        65|         40|         45|           49|Tweet fictício ge...|
|       30|             28|2023-07-14T19:01:...| 67|        37|         59|         48|           17|Este é um tweet f...|
+---------+-----

In [19]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
